In [1]:
import json
import numpy as np
import pandas as pd
import re

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
from time import sleep


In [2]:
def read_data(input_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [3]:
data = read_data('../data/key_30_exam.json')

In [4]:
keys = []
for k, v in data.items():
    keys.append(k)
    keys.extend(v)

In [5]:
keys = list(np.unique(keys))

In [6]:
df = pd.read_excel('../data/Format_benh_an.xlsx', sheet_name='Chỉ số xét nghiệm', )

In [7]:
keys.extend(list(df['Unnamed: 1']))

In [8]:
def remove_first_number(text):
    text = re.sub('^\d\d*\.', '', text).strip()
    return text

In [9]:
keys_word = []
for k in keys:
    if k == '': continue
    keys_word.append(remove_first_number(k))

TypeError: expected string or bytes-like object

In [23]:
with open('./key_word.txt', 'w', encoding='utf-8') as f:
    for item in keys_word:
        f.write('%s\n' %item)

In [12]:
keys = []
with open('./key_word.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        keys.append(line.strip())

In [3]:
keys = []
with open('../data/symptom.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        line = line.strip()
        if len(line.split()) > 1:
            keys.append(line)

In [4]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--window-size=1920x1080")

In [25]:
with open('./query_symptom_disease.json', 'r', encoding='utf-8') as f:
    urls = json.load(f)

In [26]:
len(urls)

24302

In [20]:
tmp = {}
for i in urls:
    query = i['query']
    if re.search('\?$', query):
        if query not in tmp:
            tmp[query] = i['href']

In [21]:
urls = []
for k, v in tmp.items():
    urls.append({
        'query': k,
        'href': v
    })

In [22]:
len(urls)

24302

In [24]:
with open('./query_symptom_disease.json', 'w', encoding='utf-8') as f:
    json.dump(urls, f, ensure_ascii=False)

In [18]:
browser=webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver")

# urls = []
sleep(1)
for k in tqdm(keys[683:]):
    browser.get('https://www.vinmec.com/vi/bai-viet/tim-kiem/?q=S%E1%BB%91%20l%C6%B0%E1%BB%A3ng%20h%E1%BB%93ng%20c%E1%BA%A7u')
    key_search = k
    searching_bar = browser.find_element(By.XPATH, "//div[@class='has-suggestion']")
    input_type = searching_bar.find_element(By.XPATH, "./input[@id='search-input']")
    # print(input_type)
    input_type.send_keys(Keys.CONTROL + "a")
    input_type.send_keys(Keys.DELETE)
    input_type.send_keys(k)
    input_type.send_keys(Keys.ENTER)

    try:
        pagination = browser.find_element(By.XPATH, "//div[@class='pagination']")
        pagination = pagination.find_element(By.XPATH, ".//span[@class='current']")
        total_page = pagination.text.split('/')[-1]
        url = browser.current_url
    except:
        try:
            retrieval = browser.find_element(By.XPATH, "//div[@class='content col-xs-12 col-md-8 no-paddings-sm']")
            retrieval_lists = retrieval.find_element(By.XPATH, "./ul")
            retrieval_docs = retrieval_lists.find_elements(By.XPATH, "./li")
            for doc in retrieval_docs:
                tag_doc = doc.find_element(By.XPATH, ".//h3")
                tag_doc = tag_doc.find_element(By.XPATH, ".//a")
                query = tag_doc.text
                href = tag_doc.get_attribute('href')
                urls.append({
                    'query': query,
                    'href': href
                })
            continue
        except:
            continue   
    
    for i in range(1, int(total_page)+1):
        try:
            sleep(0.5)
            curr_url = url + '&page=' + str(i)
            browser.get(curr_url)
            retrieval = browser.find_element(By.XPATH, "//div[@class='content col-xs-12 col-md-8 no-paddings-sm']")
        except:
            break
        retrieval_lists = retrieval.find_element(By.XPATH, "./ul")
        retrieval_docs = retrieval_lists.find_elements(By.XPATH, "./li")
        for doc in retrieval_docs:
            tag_doc = doc.find_element(By.XPATH, ".//h3")
            tag_doc = tag_doc.find_element(By.XPATH, ".//a")
            query = tag_doc.text
            href = tag_doc.get_attribute('href')
            urls.append({
                'query': query,
                'href': href
            })



/var/folders/d5/4zbsmpf14f12lkqkw8y5ty8r0000gn/T/ipykernel_1520/4086920663.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser=webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver")
/var/folders/d5/4zbsmpf14f12lkqkw8y5ty8r0000gn/T/ipykernel_1520/4086920663.py:1: DeprecationWarning: use options instead of chrome_options
  browser=webdriver.Chrome(chrome_options=chrome_options, executable_path="./chromedriver")
  3%|▎         | 5/175 [18:05<10:15:04, 217.09s/it]


WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.75)
Stacktrace:
0   chromedriver                        0x0000000100ef3c34 chromedriver + 4406324
1   chromedriver                        0x0000000100e8d290 chromedriver + 3986064
2   chromedriver                        0x0000000100aee71c chromedriver + 190236
3   chromedriver                        0x0000000100ade794 chromedriver + 124820
4   chromedriver                        0x0000000100adf010 chromedriver + 126992
5   chromedriver                        0x0000000100ad9fa4 chromedriver + 106404
6   chromedriver                        0x0000000100b45efc chromedriver + 548604
7   chromedriver                        0x0000000100b12864 chromedriver + 338020
8   chromedriver                        0x0000000100eb9320 chromedriver + 4166432
9   chromedriver                        0x0000000100ece748 chromedriver + 4253512
10  chromedriver                        0x0000000100ed2cbc chromedriver + 4271292
11 